# Tutorial 10 - Creating a model

In [Tutorial 9](./Tutorial%209%20-%20Changing%20the%20mesh.ipynb) we showed how to change the mesh using on of the built-in battery models in PyBaMM. In this tutorial we show how to create a simple model from scratch in PyBaMM.

As simple example, we consider the problem of linear diffusion on a unit sphere with a flux at the boundary that depends on the concentration
\begin{equation*}
  \frac{\partial c}{\partial t} = \nabla \cdot (\nabla c),
\end{equation*}
with the following boundary and initial conditions:
\begin{equation*}
  \left.\frac{\partial c}{\partial r}\right\vert_{r=0} = 0, \quad \left.\frac{\partial c}{\partial r}\right\vert_{r=1} = -j, \quad \left.c\right\vert_{t=0} = 1,
\end{equation*}
where
\begin{equation*}
j = j_0(1-c)^{1/2}c^{1/2}.
\end{equation*}
Here $j_0$ is a parameter we can control. In this example we will assume that everything is dimensionless and just focus on how to set up and solve the model rather than any specific physical interpretation.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


## Setting up the model

First we load an empty model. We use the `BaseModel` class that sets up all the basic framework on which our model will be built.

In [4]:
model = pybamm.BaseModel()

We then define our variables and parameters using the `Variable` and `Parameter` classes, respectively. Since we are solving a PDE we need to tell PyBaMM the domain each variable belongs to so that it can be discretised in space in the correct way. This is done by passing the keyword argument `domain`, and in this example we choose the domain "particle".

In [5]:
c = pybamm.Variable("Concentration", domain="negative particle")
j0 = pybamm.Parameter("Flux parameter")

We then state out governing equations. In PyBaMM we distinguish between Ordinary Differential Equations of the form $dy/dt = \text{rhs}$ and Algebraic Equations of the form $f(y) = 0$. The model equations are stored in dictionaries where the key is the variable and the value is the rhs for ODEs and the residual ($f(y)$) for algebraic equations.

Sometime it is useful to define intermediate quantities in order to express the governing equations more easily. In this example we define the flux, then define the rhs to be minus the divergence of the flux. The equation is then added to the dictionary `model.rhs`

In [6]:
N = -pybamm.grad(c)  # define the flux
dcdt = -pybamm.div(N)  # define the rhs equation

model.rhs = {c: dcdt}  # add the equation to rhs dictionary with the variable as the key 